# Connectivity Cost Estimation

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import json
import os
from ipywidgets import FileUpload, Output, Label, HBox, HTML, Layout

from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload
from giga.viz.notebooks.components.html.sections import section

import warnings
warnings.filterwarnings('ignore')


show_map = os.environ.get("FF_STATIC_MAP_NOTEBOOK", True)

output = Output()

upload = GigaFileUpload(accept='.json', multiple=True, description="Import Config")
inputs = CostEstimationParameterInput(local_data_workspace='workspace', show_map=show_map)

def on_upload_change(change):
    try:
        imported = json.loads(upload.value[0].content.tobytes())
    except IndexError:
        imported = {}
    with output:
        output.clear_output()
        inputs.update(imported)
    display(imported)
    display(upload, inputs.parameter_input(), output)

upload.observe(on_upload_change, names='value')
display(section(
    title="Configuration",
    contents=HBox([
        upload,
        HTML("For help, see documentation on <a href=\"../README.md\" style=\"text-decoration:underline\">configuring and running these models</a>."),
    ]),
    extra_class="header"
))

display(inputs.parameter_input(), output)

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Button, Output, HBox, VBox, HTML, GridBox
from tqdm import tqdm
from IPython.display import clear_output
import sys
import time
import logging
import geopandas as gpd

import json
from io import StringIO
from io import BytesIO


from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.utils.logging import LOGGER
from giga.viz.notebooks.tables import display_summary_table, plot_cost_breakdown, output_to_capex_details
from giga.viz.notebooks.helpers import output_to_table
from giga.viz.notebooks.components.html.sections import section
from giga.viz.notebooks.components.widgets.giga_buttons import make_run_button, make_run_again_button, make_show_maps_button, make_show_full_table_button
from giga.viz.notebooks.components.widgets.giga_export import make_export_button_row
from giga.viz.notebooks.components.dashboard.result_dashboard import ResultDashboard
from giga.viz.notebooks.maps import show_cost_map, show_electricity_map
from giga.viz.notebooks.fiber import plot_fiber_connections, default_map

result_output = Output()

output_space = None
data_space = None
config = None
button = None

map_output = Output()
table_output = Output()

def show_all_maps(event):
    with map_output:
        clear_output(wait=True)
        if inputs.data_parameters().school_data_conf.country_id == 'brazil':
            display(HTML("<hr><b><font color='#5b8ff0'>Unable to display maps for Brazil</b><br><br>"))
            return

        center = inputs.defaults[inputs.data_parameters().school_data_conf.country_id].data.country_center_tuple

        display(section("Electricity Map", show_electricity_map(data_space, location=center)))

        display(section("Cost Map", show_cost_map(data_space, output_space, location=center)))


        if output_space.fiber_costs is not None:
            m = default_map(location=center)
            fiber_html = HTML(
                    plot_fiber_connections(data_space.fiber_coordinates, data_space.school_coordinates, 
                                            output_space.fiber_costs.technology_results.distances, m=m)._repr_html_())
            fiber_html.add_class('folium-map')
            display(section("Fiber Map", fiber_html))

def show_full_table(event):
    with table_output:
        table = HTML(output_to_table(output_space).to_html(col_space=5, border=0))
        display(section("Cost Table", table))

def on_run_button_clicked(b):
    with result_output:
        clear_output(wait=True)
        
        global config
        global data_space
        global output_space
        # freeze and disable confgiuration + running while model in progress
        b.disabled = True
        inputs.freeze()
        # pull verbose flag
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        # setup scenario
        config = inputs.scenario_parameters()
        data_space = ModelDataSpace(inputs.data_parameters())
        data_space = data_space.filter_schools(inputs.get_selected_schools())
        output_space = OutputSpace()
        scenario = create_scenario(config, data_space, output_space)
        # run the models
        output_space = scenario.run(progress_bar=verbose)
        table = output_to_table(output_space)
        table_geo = data_space.school_outputs_to_frame(table)
        # show summary tables
        display(HTML('<br/><br/>'))
        display_summary_table(output_space, data_space)
        # start the dashboard
        dashboard = ResultDashboard(table_geo)
        dashboard.display()
        # make export + rerun buttons
        export_buttons = make_export_button_row(table, inputs)
        run_again_button = make_run_again_button(inputs, result_output, b)
        show_maps_button = make_show_maps_button(show_all_maps)
        show_maps_button.on_click(show_all_maps)
        full_table_button = make_show_full_table_button(show_full_table)
        full_table_button.on_click(show_full_table)

        # Create a grid with two columns, splitting space equally
        layout = Layout(grid_template_columns='1fr 1fr')
        display(GridBox([
            section("More", VBox([
                run_again_button,
                HTML("<hr/>"),
                show_maps_button,
                full_table_button
            ])),
            section("Export", export_buttons)
        ], layout=layout))

button = make_run_button(on_run_button_clicked)
button.on_click(on_run_button_clicked)
display(section(
    title="Run the Model",
    contents=VBox([
        button,
        HTML("<br/>"),
        result_output,
        map_output,
        table_output
    ]),
    extra_class="run"
))